### 📌 1.Load model and Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from copy import deepcopy

import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune
from google.colab import drive
drive.mount('/content/drive')
model = models.vgg11(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/第二次尝试/VGG11_C10_layer_sensitivity.pth", map_location=torch.device('cpu')))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<All keys matched successfully>

In [ ]:
# 数据预处理（VGG 使用 ImageNet 预训练，因此需归一化到 ImageNet 均值）
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 加载 CIFAR-10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


In [ ]:
# 准确率评估函数
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
print(evaluate(model))

91.31


### 2. 📌Start Unstructured prune

##### 2.1、Iternative prune:

In [ ]:
conv_layers = [(name, m) for name, m in model.features.named_modules() if isinstance(m, nn.Conv2d)]
layer_names = [name for name, _ in conv_layers]
conv_layers

[('0', Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
 ('3', Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
 ('6', Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
 ('8', Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
 ('11', Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
 ('13', Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
 ('16', Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
 ('18', Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))]

In [ ]:
#Utilize the layer sensitivity we get before
prune_rate = [0.1,0.4,0.5,0.5,0.6,0.6,0.5,0.7]

In [ ]:
for (name, layer), amt in zip(conv_layers, prune_rate):
    prune.l1_unstructured(layer, name='weight', amount=amt)
    prune.remove(layer,"weight")

In [ ]:
#Calculate Sparity After Iterative pruning
def calculate_model_sparsity(model):
    total_params = 0
    zero_params = 0

    for name, param in model.named_parameters():
        if "weight" in name and param.requires_grad:
            num_params = param.numel()
            num_zeros = torch.sum(param == 0).item()
            total_params += num_params
            zero_params += num_zeros
            layer_sparsity = 100.0 * num_zeros / num_params
            print(f"{name:40} | Sparsity: {layer_sparsity:.2f}% ({num_zeros}/{num_params})")

    total_sparsity = 100.0 * zero_params / total_params
    print(f"\n🔍 Total model sparsity: {total_sparsity:.2f}% ({zero_params}/{total_params})")

    return total_sparsity
# get the sparity
sparsity = calculate_model_sparsity(model)


features.0.weight                        | Sparsity: 10.01% (173/1728)
features.3.weight                        | Sparsity: 40.00% (29491/73728)
features.6.weight                        | Sparsity: 50.00% (147456/294912)
features.8.weight                        | Sparsity: 50.00% (294912/589824)
features.11.weight                       | Sparsity: 60.00% (707789/1179648)
features.13.weight                       | Sparsity: 60.00% (1415578/2359296)
features.16.weight                       | Sparsity: 50.00% (1179648/2359296)
features.18.weight                       | Sparsity: 70.00% (1651507/2359296)
classifier.0.weight                      | Sparsity: 0.00% (0/102760448)
classifier.3.weight                      | Sparsity: 0.00% (0/16777216)
classifier.6.weight                      | Sparsity: 0.00% (0/40960)

🔍 Total model sparsity: 4.21% (5426554/128796352)


In [ ]:
print(evaluate(model))

88.06


##### 2.2、Global pruning:
- Each layer's Sensitivity is [0.1,0.4,0.5,0.5,0.6,0.6,0.5,0.7], when we do Global pruning, we directly select 0.7 as global prune

In [ ]:
# 选择参与剪枝的模块类型（通常是 Conv2d 和 Linear）
modules_to_prune = []
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        modules_to_prune.append((module, 'weight'))  # 剪枝参数通常是 weight


In [ ]:
prune.global_unstructured(
    modules_to_prune,
    pruning_method=prune.L1Unstructured,  # 可选 L1Unstructured 或 RandomUnstructured
    amount=0.7  # 全局剪枝 70% 的参数
)

In [ ]:
for module, name in modules_to_prune:
    prune.remove(module, name)

In [ ]:
#Calculate Sparity After global pruning
def calculate_model_sparsity(model):
    total_params = 0
    zero_params = 0

    for name, param in model.named_parameters():
        if "weight" in name and param.requires_grad:
            num_params = param.numel()
            num_zeros = torch.sum(param == 0).item()
            total_params += num_params
            zero_params += num_zeros
            layer_sparsity = 100.0 * num_zeros / num_params
            print(f"{name:40} | Sparsity: {layer_sparsity:.2f}% ({num_zeros}/{num_params})")

    total_sparsity = 100.0 * zero_params / total_params
    print(f"\n🔍 Total model sparsity: {total_sparsity:.2f}% ({zero_params}/{total_params})")

    return total_sparsity
# get the sparity
sparsity = calculate_model_sparsity(model)


features.0.weight                        | Sparsity: 2.43% (42/1728)
features.3.weight                        | Sparsity: 17.81% (13131/73728)
features.6.weight                        | Sparsity: 23.00% (67842/294912)
features.8.weight                        | Sparsity: 26.26% (154869/589824)
features.11.weight                       | Sparsity: 30.03% (354247/1179648)
features.13.weight                       | Sparsity: 35.53% (838221/2359296)
features.16.weight                       | Sparsity: 34.36% (810612/2359296)
features.18.weight                       | Sparsity: 35.60% (839893/2359296)
classifier.0.weight                      | Sparsity: 75.67% (77754423/102760448)
classifier.3.weight                      | Sparsity: 55.45% (9302496/16777216)
classifier.6.weight                      | Sparsity: 52.91% (21670/40960)

🔍 Total model sparsity: 70.00% (90157446/128796352)


In [ ]:
print(evaluate(model))

91.3


##### 2.3 Fine-tunning

In [ ]:
#Fine tuning Iternative prune model
model_finetunning = deepcopy(model)
model_finetunning = model_finetunning.to(device)

# 可选：冻结特征提取部分
for param in model_finetunning.features.parameters():
    param.requires_grad = False

# -----------------------
# 3. 定义损失和优化器
# -----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_finetunning.parameters(), lr=0.0001)

# -----------------------
# 4. 模型训练
# -----------------------
for epoch in range(5):  # 可根据需要改epoch数
    model_finetunning.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_finetunning(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个mini-batch打印一次loss
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("Finished Training")

[Epoch 1, Batch 100] loss: 0.069
[Epoch 1, Batch 200] loss: 0.062
[Epoch 1, Batch 300] loss: 0.072
[Epoch 1, Batch 400] loss: 0.073
[Epoch 1, Batch 500] loss: 0.067
[Epoch 1, Batch 600] loss: 0.080
[Epoch 1, Batch 700] loss: 0.071
[Epoch 2, Batch 100] loss: 0.040
[Epoch 2, Batch 200] loss: 0.039
[Epoch 2, Batch 300] loss: 0.041
[Epoch 2, Batch 400] loss: 0.045
[Epoch 2, Batch 500] loss: 0.055
[Epoch 2, Batch 600] loss: 0.042
[Epoch 2, Batch 700] loss: 0.049
[Epoch 3, Batch 100] loss: 0.027
[Epoch 3, Batch 200] loss: 0.026
[Epoch 3, Batch 300] loss: 0.032
[Epoch 3, Batch 400] loss: 0.033
[Epoch 3, Batch 500] loss: 0.031
[Epoch 3, Batch 600] loss: 0.030
[Epoch 3, Batch 700] loss: 0.038
[Epoch 4, Batch 100] loss: 0.030
[Epoch 4, Batch 200] loss: 0.024
[Epoch 4, Batch 300] loss: 0.032
[Epoch 4, Batch 400] loss: 0.025
[Epoch 4, Batch 500] loss: 0.035
[Epoch 4, Batch 600] loss: 0.038
[Epoch 4, Batch 700] loss: 0.025
[Epoch 5, Batch 100] loss: 0.016
[Epoch 5, Batch 200] loss: 0.016
[Epoch 5, 

In [ ]:
#Re-evaluate
print(evaluate(model_finetunning))

90.06


In [ ]:
# 5919132/128796352

0.045957295436442176

In [ ]:
#calculate Sparsity
sparsity = calculate_model_sparsity(model_finetunning)


features.0.weight                        | Sparsity: 10.01% (173/1728)
features.3.weight                        | Sparsity: 40.00% (29491/73728)
features.6.weight                        | Sparsity: 50.00% (147456/294912)
features.8.weight                        | Sparsity: 50.00% (294912/589824)
features.11.weight                       | Sparsity: 60.00% (707789/1179648)
features.13.weight                       | Sparsity: 60.00% (1415578/2359296)
features.16.weight                       | Sparsity: 50.00% (1179648/2359296)
features.18.weight                       | Sparsity: 70.00% (1651507/2359296)
classifier.0.weight                      | Sparsity: 0.00% (0/102760448)
classifier.3.weight                      | Sparsity: 0.00% (0/16777216)
classifier.6.weight                      | Sparsity: 0.00% (0/40960)

🔍 Total model sparsity: 4.21% (5426554/128796352)


In [ ]:
#####  Fine-tunning global prune model
model_finetunning = deepcopy(model)
model_finetunning = model_finetunning.to(device)

# 可选：冻结特征提取部分
for param in model_finetunning.features.parameters():
    param.requires_grad = False

# -----------------------
# 3. 定义损失和优化器
# -----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_finetunning.parameters(), lr=0.0001)

# -----------------------
# 4. 模型训练
# -----------------------
for epoch in range(5):  # 可根据需要改epoch数
    model_finetunning.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_finetunning(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个mini-batch打印一次loss
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("Finished Training")

[Epoch 1, Batch 100] loss: 0.054
[Epoch 1, Batch 200] loss: 0.047
[Epoch 1, Batch 300] loss: 0.045
[Epoch 1, Batch 400] loss: 0.047
[Epoch 1, Batch 500] loss: 0.045
[Epoch 1, Batch 600] loss: 0.052
[Epoch 1, Batch 700] loss: 0.055
[Epoch 2, Batch 100] loss: 0.023
[Epoch 2, Batch 200] loss: 0.027
[Epoch 2, Batch 300] loss: 0.035
[Epoch 2, Batch 400] loss: 0.026
[Epoch 2, Batch 500] loss: 0.031
[Epoch 2, Batch 600] loss: 0.035
[Epoch 2, Batch 700] loss: 0.027
[Epoch 3, Batch 100] loss: 0.017
[Epoch 3, Batch 200] loss: 0.021
[Epoch 3, Batch 300] loss: 0.020
[Epoch 3, Batch 400] loss: 0.024
[Epoch 3, Batch 500] loss: 0.026
[Epoch 3, Batch 600] loss: 0.023
[Epoch 3, Batch 700] loss: 0.029
[Epoch 4, Batch 100] loss: 0.012
[Epoch 4, Batch 200] loss: 0.015
[Epoch 4, Batch 300] loss: 0.012
[Epoch 4, Batch 400] loss: 0.018
[Epoch 4, Batch 500] loss: 0.014
[Epoch 4, Batch 600] loss: 0.022
[Epoch 4, Batch 700] loss: 0.017
[Epoch 5, Batch 100] loss: 0.009
[Epoch 5, Batch 200] loss: 0.006
[Epoch 5, 

In [ ]:
#the result is after gloabal pruning and fine-tunning
print(evaluate(model_finetunning))

92.89


In [ ]:
#calculate Sparsity
sparsity = calculate_model_sparsity(model_finetunning)

classifier.0.weight                      | Sparsity: 0.36% (374966/102760448)
classifier.3.weight                      | Sparsity: 0.70% (117577/16777216)
classifier.6.weight                      | Sparsity: 0.09% (35/40960)

🔍 Total model sparsity: 0.41% (492578/119578624)


### 3. 📌Start Structured prune

In [ ]:
model = models.vgg11(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/第二次尝试/VGG11_C10_layer_sensitivity.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
prune_rates = [0.1, 0.4, 0.5, 0.5, 0.6, 0.6, 0.5, 0.7]
conv_idx = 0
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        rate = prune_rates[conv_idx]
        prune.ln_structured(module, name='weight', amount=rate, n=2, dim=0)
        prune.remove(module, 'weight')
        print(f"Pruned {name} with rate {rate}")
        conv_idx += 1
        if conv_idx >= len(prune_rates):
            break  # 避免下标越界

Pruned features.0 with rate 0.1
Pruned features.3 with rate 0.4
Pruned features.6 with rate 0.5
Pruned features.8 with rate 0.5
Pruned features.11 with rate 0.6
Pruned features.13 with rate 0.6
Pruned features.16 with rate 0.5
Pruned features.18 with rate 0.7


In [ ]:
print(evaluate(model))

10.0


In [ ]:
model_finetunning = deepcopy(model)
model_finetunning = model_finetunning.to(device)

# 可选：冻结特征提取部分
for param in model_finetunning.features.parameters():
    param.requires_grad = False

# -----------------------
# 3. 定义损失和优化器
# -----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_finetunning.parameters(), lr=0.0001)

# -----------------------
# 4. 模型训练
# -----------------------
for epoch in range(3):  # 可根据需要改epoch数
    model_finetunning.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_finetunning(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个mini-batch打印一次loss
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("Finished Re-Training")

[Epoch 1, Batch 100] loss: 2.647
[Epoch 1, Batch 200] loss: 2.305
[Epoch 1, Batch 300] loss: 2.301
[Epoch 1, Batch 400] loss: 2.290
[Epoch 1, Batch 500] loss: 2.269
[Epoch 1, Batch 600] loss: 2.246
[Epoch 1, Batch 700] loss: 2.249
[Epoch 2, Batch 100] loss: 2.213
[Epoch 2, Batch 200] loss: 2.212
[Epoch 2, Batch 300] loss: 2.207
[Epoch 2, Batch 400] loss: 2.207
[Epoch 2, Batch 500] loss: 2.193
[Epoch 2, Batch 600] loss: 2.187
[Epoch 2, Batch 700] loss: 2.187
[Epoch 3, Batch 100] loss: 2.176
[Epoch 3, Batch 200] loss: 2.161
[Epoch 3, Batch 300] loss: 2.153
[Epoch 3, Batch 400] loss: 2.187
[Epoch 3, Batch 500] loss: 2.174
[Epoch 3, Batch 600] loss: 2.162
[Epoch 3, Batch 700] loss: 2.160
Finished Re-Training


In [ ]:
#稀疏度计算
def calculate_model_sparsity(model):
    total_params = 0
    zero_params = 0

    for name, param in model.named_parameters():
        if "weight" in name:
            num_params = param.numel()
            num_zeros = torch.sum(param == 0).item()
            total_params += num_params
            zero_params += num_zeros
            layer_sparsity = 100.0 * num_zeros / num_params
            print(f"{name:40} | Sparsity: {layer_sparsity:.2f}% ({num_zeros}/{num_params})")

    total_sparsity = 100.0 * zero_params / total_params
    print(f"\n🔍 Total model sparsity: {total_sparsity:.2f}% ({zero_params}/{total_params})")

    return total_sparsity
# 假设你有一个剪枝后的模型
sparsity = calculate_model_sparsity(model_finetunning)

features.0.weight                        | Sparsity: 9.38% (162/1728)
features.3.weight                        | Sparsity: 39.84% (29376/73728)
features.6.weight                        | Sparsity: 50.00% (147456/294912)
features.8.weight                        | Sparsity: 50.00% (294912/589824)
features.11.weight                       | Sparsity: 59.96% (707328/1179648)
features.13.weight                       | Sparsity: 59.96% (1414656/2359296)
features.16.weight                       | Sparsity: 50.00% (1179648/2359296)
features.18.weight                       | Sparsity: 69.92% (1649664/2359296)
classifier.0.weight                      | Sparsity: 0.00% (0/102760448)
classifier.3.weight                      | Sparsity: 0.00% (0/16777216)
classifier.6.weight                      | Sparsity: 0.00% (0/40960)

🔍 Total model sparsity: 4.21% (5423202/128796352)


In [ ]:
print(evaluate(model_finetunning))

21.15


### 4. 📌Just ignore the following code【for personal use】

In [ ]:
from torchvision import models
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from copy import deepcopy
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import torchvision
import torchvision.transforms as transforms

# 数据预处理：resize 和 normalization
transform = transforms.Compose([
    transforms.Resize(224),  # 因为 VGG 输入是 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载训练集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# 加载测试集
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:13<00:00, 12.9MB/s]


In [ ]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# -----------------------
# 2. 加载并修改预训练模型
# -----------------------
model = models.vgg11(pretrained=True)

# 替换最后一层：1000 -> 10
model.classifier[6] = nn.Linear(4096, 10)

# 将模型放到 GPU（如有）
model = model.to(device)

# 可选：冻结特征提取部分
for param in model.features.parameters():
    param.requires_grad = False

# -----------------------
# 3. 定义损失和优化器
# -----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# -----------------------
# 4. 模型训练
# -----------------------
for epoch in range(5):  # 可根据需要改epoch数
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个mini-batch打印一次loss
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("Finished Training")

[Epoch 1, Batch 100] loss: 1.010
[Epoch 1, Batch 200] loss: 0.671
[Epoch 1, Batch 300] loss: 0.580
[Epoch 1, Batch 400] loss: 0.548
[Epoch 1, Batch 500] loss: 0.485
[Epoch 1, Batch 600] loss: 0.462
[Epoch 1, Batch 700] loss: 0.463
[Epoch 2, Batch 100] loss: 0.275
[Epoch 2, Batch 200] loss: 0.254
[Epoch 2, Batch 300] loss: 0.279
[Epoch 2, Batch 400] loss: 0.256
[Epoch 2, Batch 500] loss: 0.283
[Epoch 2, Batch 600] loss: 0.274
[Epoch 2, Batch 700] loss: 0.281
[Epoch 3, Batch 100] loss: 0.138
[Epoch 3, Batch 200] loss: 0.115
[Epoch 3, Batch 300] loss: 0.126
[Epoch 3, Batch 400] loss: 0.124
[Epoch 3, Batch 500] loss: 0.122
[Epoch 3, Batch 600] loss: 0.132
[Epoch 3, Batch 700] loss: 0.134
[Epoch 4, Batch 100] loss: 0.061
[Epoch 4, Batch 200] loss: 0.061
[Epoch 4, Batch 300] loss: 0.067
[Epoch 4, Batch 400] loss: 0.076
[Epoch 4, Batch 500] loss: 0.077
[Epoch 4, Batch 600] loss: 0.080
[Epoch 4, Batch 700] loss: 0.085
[Epoch 5, Batch 100] loss: 0.047
[Epoch 5, Batch 200] loss: 0.043
[Epoch 5, 

In [ ]:
# -----------------------
# 5. 模型测试
# -----------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 85.79%


In [ ]:
# 准确率评估函数
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total
print(evaluate(model))

85.79


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 定义保存路径
save_path = "/content/drive/MyDrive/模型/VGG11_CIFAR.pth"
# 保存模型参数
torch.save(model.state_dict(), save_path)

Mounted at /content/drive


In [ ]:
# -----------------------
# 2. 加载并修改预训练模型
# -----------------------
model = models.vgg13(pretrained=True)

# 替换最后一层：1000 -> 10
model.classifier[6] = nn.Linear(4096, 10)

# 将模型放到 GPU（如有）
model = model.to(device)

# 可选：冻结特征提取部分
for param in model.features.parameters():
    param.requires_grad = False

# -----------------------
# 3. 定义损失和优化器
# -----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# -----------------------
# 4. 模型训练
# -----------------------
for epoch in range(10):  # 可根据需要改epoch数
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个mini-batch打印一次loss
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("Finished Training")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG13_Weights.IMAGENET1K_V1`. You can also use `weights=VGG13_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg13-19584684.pth" to /root/.cache/torch/hub/checkpoints/vgg13-19584684.pth
100%|██████████| 508M/508M [00:02<00:00, 224MB/s]


[Epoch 1, Batch 100] loss: 1.012
[Epoch 1, Batch 200] loss: 0.675
[Epoch 1, Batch 300] loss: 0.567
[Epoch 1, Batch 400] loss: 0.539
[Epoch 1, Batch 500] loss: 0.495
[Epoch 1, Batch 600] loss: 0.512
[Epoch 1, Batch 700] loss: 0.479
[Epoch 2, Batch 100] loss: 0.265
[Epoch 2, Batch 200] loss: 0.256
[Epoch 2, Batch 300] loss: 0.272
[Epoch 2, Batch 400] loss: 0.269
[Epoch 2, Batch 500] loss: 0.260
[Epoch 2, Batch 600] loss: 0.265
[Epoch 2, Batch 700] loss: 0.264
[Epoch 3, Batch 100] loss: 0.112
[Epoch 3, Batch 200] loss: 0.108
[Epoch 3, Batch 300] loss: 0.109
[Epoch 3, Batch 400] loss: 0.125
[Epoch 3, Batch 500] loss: 0.127
[Epoch 3, Batch 600] loss: 0.121
[Epoch 3, Batch 700] loss: 0.135
[Epoch 4, Batch 100] loss: 0.058
[Epoch 4, Batch 200] loss: 0.058
[Epoch 4, Batch 300] loss: 0.057
[Epoch 4, Batch 400] loss: 0.054
[Epoch 4, Batch 500] loss: 0.052
[Epoch 4, Batch 600] loss: 0.068
[Epoch 4, Batch 700] loss: 0.073
[Epoch 5, Batch 100] loss: 0.045
[Epoch 5, Batch 200] loss: 0.041
[Epoch 5, 

In [ ]:
# -----------------------
# 5. 模型测试
# -----------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 86.91%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 定义保存路径
save_path = "/content/drive/MyDrive/模型/VGG13_CIFAR.pth"
# 保存模型参数
torch.save(model.state_dict(), save_path)

Mounted at /content/drive


In [ ]:
# -----------------------
# 2. 加载并修改预训练模型
# -----------------------
model = models.vgg16(pretrained=True)

# 替换最后一层：1000 -> 10
model.classifier[6] = nn.Linear(4096, 10)

# 将模型放到 GPU（如有）
model = model.to(device)

# 可选：冻结特征提取部分
for param in model.features.parameters():
    param.requires_grad = False

# -----------------------
# 3. 定义损失和优化器
# -----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# -----------------------
# 4. 模型训练
# -----------------------
for epoch in range(10):  # 可根据需要改epoch数
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个mini-batch打印一次loss
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("Finished Training")

In [ ]:
# -----------------------
# 5. 模型测试
# -----------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 86.57%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 定义保存路径
save_path = "/content/drive/MyDrive/模型/VGG16_CIFAR.pth"
# 保存模型参数
torch.save(model.state_dict(), save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -----------------------
# 2. 加载并修改预训练模型
# -----------------------
model = models.vgg19(pretrained=True)

# 替换最后一层：1000 -> 10
model.classifier[6] = nn.Linear(4096, 10)

# 将模型放到 GPU（如有）
model = model.to(device)

# 可选：冻结特征提取部分
for param in model.features.parameters():
    param.requires_grad = False

# -----------------------
# 3. 定义损失和优化器
# -----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# -----------------------
# 4. 模型训练
# -----------------------
for epoch in range(10):  # 可根据需要改epoch数
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每100个mini-batch打印一次loss
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("Finished Training")

In [ ]:
# -----------------------
# 5. 模型测试
# -----------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 87.07%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 定义保存路径
save_path = "/content/drive/MyDrive/模型/VGG19_CIFAR.pth"
# 保存模型参数
torch.save(model.state_dict(), save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
